In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings(action='ignore')
# flask
import base64
import io

In [2]:
import importlib
import import_ipynb
import Utils

importing Jupyter notebook from Utils.ipynb


In [3]:
importlib.reload(Utils)
from Utils import view_model, normal, renormal, load_model, dixon_q_test

importing Jupyter notebook from Utils.ipynb


In [4]:
from flask import Flask, request
from flask import render_template, redirect, url_for
app = Flask(__name__)

import models
from models import CNN, LSTM, GRU, RNN

importing Jupyter notebook from models.ipynb


In [ ]:
@app.route('/', methods=['GET', 'POST'])
def hello():
    if request.method == 'POST':
        w_id = request.form.get('id')
        period = request.form.get('period')
        op = request.form.get('op')
        
        if not w_id or not period:
            return render_template("layout.html")

        w_id = int(w_id)  # w_id를 정수형으로 변환
        
        if op == '0':
            if w_id == 0:
                df = pd.read_csv('./csv/han.csv')
            elif w_id == 1:
                df = pd.read_csv('./csv/gum.csv')
            elif w_id == 2:
                df = pd.read_csv('./csv/nak.csv')
            elif w_id == 3:
                df = pd.read_csv('./csv/yong.csv')
        elif op == '1': # not satisfiy
            if w_id == 0:
                df = pd.read_csv('./csv/han_f.csv')
            elif w_id == 1:
                df = pd.read_csv('./csv/gum_f.csv')
            elif w_id == 2:
                df = pd.read_csv('./csv/nak_f.csv')
            elif w_id == 3:
                df = pd.read_csv('./csv/yong_f.csv')
        else:
            return render_template("layout.html")

        df = df[df['day'] == int(period)]
        
        templateData = {'col' : df['col'], 'day' : str(period), 'id': str(w_id), 'op': op}

        return render_template("layout.html", **templateData)
    else:
        return render_template("layout.html")

@app.route('/map')
def mapping():
    return render_template("map.html")

@app.route('/predict', methods=['POST'])
def predict1():
    w_id = request.form.get('id')
    period = request.form.get('period')
    ft = request.form.get('ft')
    op = request.form.get('op')
    t_set = request.form.get('t_set')
    
    w_id = int(w_id)
    period = int(period)
    
    # 모델 view
    Data = pd.read_csv('./testset/origin/g_linear_amos_rtwqi'+str(w_id)+'.csv')[365*4*24:]
    Data['m_date'] = pd.to_datetime(Data['m_date'], format='%Y-%m-%d %H:%M', errors='raise')
    Data['d2'] = Data['m_date'].dt.strftime('%Y-%m-%d')
    w_list = [['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'clola-a', 'ss', 'tn', 'flow_rate', 
               'tp', 'cod', 'a_water_depth', 'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 
               'a_dew_point', 'rtwqi', 'd2'], 
              ['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'tn', 'tp', 'clola-a', 'a_water_depth',
                'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 'a_dew_point', 'rtwqi', 'd2'],
              ['temp', 'ph', 'ec', 'do', 'toc', 'tn', 'tp', 'clola-a', 'a_water_depth',
                'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 'a_dew_point', 'rtwqi', 'd2'], 
              ['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'tn', 'tp', 'clola-a', 'w_depth', 'a_water_depth',
                'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 'a_dew_point','rtwqi', 'd2']]
    #일별 예측이 목표이므로, 일별 평균으로 변환한다.    
    Data2 = Data[w_list[w_id]].groupby('d2').mean()
    Data2 = Data2.reset_index()
    Data_n = pd.DataFrame(normal(Data2[Data2.columns[1:]]), columns = Data2.columns[1:])
    
    # 데이터 슬라이싱
    days = period
    if days == 1:
        window_size = 5 # 5일치 데이터로 학습
    elif days == 5:
        window_size = 20
    elif days == 7:
        window_size = 14
    elif days == 14:
        window_size = 28

    # 1.1~1.5일 train / 1.6 test부터 시작
    X = []
    Y = []
    for i in range(len(Data_n)-window_size-days):
        X.append(Data_n[i:i+window_size])
        Y.append(Data_n[i+window_size:i+window_size+days][ft])
    X2 = np.array(X)
    Y2 = np.array(Y)
    X2 = torch.tensor(X2, dtype=torch.float32)
    Y2 = torch.tensor(Y2, dtype=torch.float32)
    
    o_path = './best_model/'
    
    if op == '0':
        if w_id == 0:
            df = pd.read_csv('./csv/han.csv')
        elif w_id == 1:
            df = pd.read_csv('./csv/gum.csv')
        elif w_id == 2:
            df = pd.read_csv('./csv/nak.csv')
        elif w_id == 3:
            df = pd.read_csv('./csv/yong.csv')
    elif op == '1': # not satisfiy
        o_path = './fail_model/'
        if w_id == 0:
            df = pd.read_csv('./csv/han_f.csv')
        elif w_id == 1:
            df = pd.read_csv('./csv/gum_f.csv')
        elif w_id == 2:
            df = pd.read_csv('./csv/nak_f.csv')
        elif w_id == 3:
            df = pd.read_csv('./csv/yong_f.csv')
        
    # 모델종류    
    md = df[(df['day'] == period) & (df['col'] == ft)]['model'].values[0]
    
    model = load_model(o_path+'water'+str(w_id+1)+'/', str(period)+'_'+ft)
    
    #view
    loss_fn = nn.MSELoss()
    if md == 'c':
        output = model(X2)
    else:
        output = model(X2, days)
    nse = 1. - torch.sum(torch.square(Y2-output))/torch.sum(torch.square(Y2-torch.mean(Y2)))
    mse = mean_squared_error(Y2.cpu().detach().numpy(), output.cpu().detach().numpy())
    rmse = np.sqrt(mse)


    t1 = output.cpu().detach().numpy()
    t2 = Y2.cpu().detach().numpy()

    show_predict = pd.DataFrame({'y': t2[0].reshape(-1), 'pred': t1[0].reshape(-1)})
    for i in range(1, len(t1)):
        nw = {'y': t2[i][-1], 'pred': t1[i][-1]}
        show_predict = show_predict.append(nw, ignore_index=True)

    show_predict = renormal(show_predict, [Data2[ft].min(), Data2[ft].max()])
    show_predict['d2'] = Data2['d2'][window_size:-1].values

    plt.figure(figsize=(15,6))
    plt.title('prediction graph:'+ft)
    plt.plot(show_predict['y'], label='origin')
    plt.plot(show_predict['pred'], label='predict')
    plt.legend()
    
    #plt save
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    plot_data = base64.b64encode(buffer.read()).decode()
    
    templateData = {'data': show_predict[:period], 'col' : ft, 'day' : str(period), 'id': str(w_id), 'md': md, 'loss': round(loss_fn(output, Y2).item(), 2), 'rmse': round(rmse, 2), 'nse': round(nse.item(), 2), 'plot_data': plot_data, 't_set': t_set}
    
    return render_template("predict.html",  **templateData)

#2023. 3월 데이터로 예측
@app.route('/predict2', methods=['POST'])
def predict2():
    w_id = request.form.get('id')
    period = request.form.get('period')
    ft = request.form.get('ft')
    op = request.form.get('op')
    t_set = request.form.get('t_set')
    
    w_id = int(w_id)
    period = int(period)
    
    w_list = [['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'clola-a', 'ss', 'tn', 'flow_rate', 
               'tp', 'cod', 'a_water_depth', 'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 
               'a_dew_point', 'rtwqi', 'd2'], 
              ['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'tn', 'tp', 'clola-a', 'a_water_depth',
                'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 'a_dew_point', 'rtwqi', 'd2'],
              ['temp', 'ph', 'ec', 'do', 'toc', 'tn', 'tp', 'clola-a', 'a_water_depth',
                'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 'a_dew_point', 'rtwqi', 'd2'], 
              ['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'tn', 'tp', 'clola-a', 'w_depth', 'a_water_depth',
                'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 'a_dew_point','rtwqi', 'd2']]

    # 데이터 가공
    t1 = pd.read_csv('./testset/new/auto_water'+str(w_id)+'.csv')[:30*24]
    t2 = pd.read_csv('./testset/new/asos_water'+str(w_id)+'.csv', encoding='cp949')
    t4 = pd.read_csv('./testset/new/m_water'+str(w_id)+'.csv', encoding='cp949')
    
    #수질측정망 가공
    t4 = t4[['년/월/일', 'SS(㎎/L)', 'TN(㎎/L)', '유량(㎥/s)', '수심(m)', 'TP(㎎/L)', 'COD(㎎/L)']]
    t4.columns = ['m_date', 'ss', 'tn', 'flow_rate', 'w_depth', 'tp', 'cod']
    t4['m_date'] = pd.to_datetime(t4['m_date'], format='%Y-%m-%d %H:%M', errors='raise')
    
    # 수계에 따른 feature get
    if w_id == 0:
        f = w_list[w_id][:7]
        t4 = t4[['m_date', 'ss', 'tn', 'flow_rate', 'tp', 'cod']]
    elif w_id == 1:
        f = w_list[w_id][:9]
        t1['tp'].replace('ND', 0, inplace=True)
        t1['tp'] = t1['tp'].astype(float)
    elif w_id == 2:  
        f = w_list[w_id][:8]
        t1['tp'].replace('ND', 0, inplace=True)
        t1['tp'] = t1['tp'].astype(float)
    elif w_id == 3:
        f = w_list[w_id][:9]
        t1['tp'].replace('ND', 0, inplace=True)
        t1['tp'] = t1['tp'].astype(float)
        t4 = t4[['m_date', 'w_depth']]
    f.append('m_date')
    f.append('rtwqi')
    f.append('g_rtwqi')

    t1 = t1[f]

    # 여기서 q-test로 이상치처리 데이터 학습을 시도
    t3 = t1.copy() # 원본데이터 저장 #
    t1 = t1.interpolate()
    t1 = dixon_q_test(t1) #이상치 처리
    t1 = t1.interpolate()
    t1['m_date'] = t1['m_date'].str.replace('h', ':00', 1)
    t1['m_date'] = pd.to_datetime(t1['m_date'], format='%Y-%m-%d %H:%M', errors='raise')
    t3['m_date'] = t3['m_date'].str.replace('h', ':00', 1) #
    t3['m_date'] = pd.to_datetime(t3['m_date'], format='%Y-%m-%d %H:%M', errors='raise') #
    t2 = t2[['일시', '강수량(mm)', '습도(%)', '현지기압(hPa)', '해면기압(hPa)', '적설(cm)', '이슬점온도(°C)']]
    t2.columns = ['m_date', 'a_water_depth', 'a_humidity', 'a_hpa', 'a_whpa', 'a_snow', 'a_dew_point']
    t2[['a_water_depth', 'a_snow']] = t2[['a_water_depth', 'a_snow']].fillna(0)
    t2 = t2.interpolate()
    t2['m_date'] = pd.to_datetime(t2['m_date'], format='%Y-%m-%d %H:%M', errors='raise')
    
    Data = pd.merge(t1, t2, on='m_date', how='inner')
    if w_id in [0, 3]:
        Data = pd.merge(Data, t4, how='left', on='m_date')
        Data[Data.columns[-5:]] = Data[Data.columns[-5:]].interpolate()
        Data[Data.columns[-5:]] = Data[Data.columns[-5:]].interpolate(method='bfill')
    
    Data['d2'] = Data['m_date'].dt.strftime('%Y-%m-%d')
    Data2 = Data[w_list[w_id]].groupby('d2').mean()
    Data2 = Data2.reset_index()
    Data_n = pd.DataFrame(normal(Data2[Data2.columns[1:]]), columns = Data2.columns[1:])

    #t3 처리
    t3['d2'] = t3['m_date'].dt.strftime('%Y-%m-%d')
    t3 = t3.groupby('d2').mean()
    t3 = t3.reset_index()

    # 데이터 슬라이싱
    days = period
    if days == 1:
        window_size = 5 # 5일치 데이터로 학습
    elif days == 5:
        window_size = 20
    elif days == 7:
        window_size = 14
    elif days == 14:
        window_size = 28

    # 1.1~1.5일 train / 1.6 test부터 시작
    X = []
    Y = []
    for i in range(len(Data_n)-window_size-days):
        X.append(Data_n[i:i+window_size])
        Y.append(Data_n[i+window_size:i+window_size+days][ft])
    X2 = np.array(X)
    Y2 = np.array(Y)
    X2 = torch.tensor(X2, device="cpu", dtype=torch.float32)
    Y2 = torch.tensor(Y2, device="cpu", dtype=torch.float32)
    
    o_path = './best_model/'
    
    if op == '0':
        if w_id == 0:
            df = pd.read_csv('./csv/han.csv')
        elif w_id == 1:
            df = pd.read_csv('./csv/gum.csv')
        elif w_id == 2:
            df = pd.read_csv('./csv/nak.csv')
        elif w_id == 3:
            df = pd.read_csv('./csv/yong.csv')
    elif op == '1': # not satisfiy
        o_path = './fail_model/'
        if w_id == 0:
            df = pd.read_csv('./csv/han_f.csv')
        elif w_id == 1:
            df = pd.read_csv('./csv/gum_f.csv')
        elif w_id == 2:
            df = pd.read_csv('./csv/nak_f.csv')
        elif w_id == 3:
            df = pd.read_csv('./csv/yong_f.csv')


    # 모델종류    
    md = df[(df['day'] == period) & (df['col'] == ft)]['model'].values[0]
    
    model = load_model(o_path+'water'+str(w_id+1)+'/', str(period)+'_'+ft)
    
    #view
    loss_fn = nn.MSELoss()
    if md == 'c':
        output = model(X2)
    else:
        output = model(X2, days)
    nse = 1. - torch.sum(torch.square(Y2-output))/torch.sum(torch.square(Y2-torch.mean(Y2)))
    mse = mean_squared_error(Y2.cpu().detach().numpy(), output.cpu().detach().numpy())
    rmse = np.sqrt(mse)

    t1 = output.cpu().detach().numpy()
    t2 = Y2.cpu().detach().numpy()

    show_predict = pd.DataFrame({'y': t2[0].reshape(-1), 'pred': t1[0].reshape(-1)})
    for i in range(1, len(t1)):
        nw = {'y': t2[i][-1], 'pred': t1[i][-1]}
        show_predict = show_predict.append(nw, ignore_index=True)

    show_predict = renormal(show_predict, [Data2[ft].min(), Data2[ft].max()])
    if ft in f:
        show_predict['origin'] = t3.iloc[window_size:-1][ft].values
    show_predict['d2'] = Data2['d2'][window_size:-1].values
    plt.figure(figsize=(15,6))
    plt.title('prediction graph:'+ft)
    if ft in f:
        plt.plot(show_predict['origin'], label='origin')
    plt.plot(show_predict['y'], label='q_test_origin')
    plt.plot(show_predict['pred'], label='predict')
    plt.legend()
    
    #plt save
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    plot_data = base64.b64encode(buffer.read()).decode()
    
    
    templateData = {'data': show_predict[:period], 'col' : ft, 'day' : str(period), 'id': str(w_id), 'md': md, 'loss': round(loss_fn(output, Y2).item(), 2), 'rmse': round(rmse, 2), 'nse': round(nse.item(), 2), 'plot_data': plot_data, 't_set': t_set}
    
    return render_template("predict.html",  **templateData)


if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.124:5000/ (Press CTRL+C to quit)
192.168.0.11 - - [13/May/2023 00:27:20] "GET / HTTP/1.1" 200 -
192.168.0.11 - - [13/May/2023 00:27:21] "GET /map HTTP/1.1" 200 -
192.168.0.11 - - [13/May/2023 00:27:27] "POST / HTTP/1.1" 200 -
192.168.0.11 - - [13/May/2023 00:27:27] "GET /map HTTP/1.1" 200 -
192.168.0.11 - - [13/May/2023 00:27:32] "POST /predict2 HTTP/1.1" 200 -
[2023-05-13 00:27:41,661] ERROR in app: Exception on /predict2 [POST]
Traceback (most recent call last):
  File "/home/coinyawong/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/coinyawong/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/coinyawong/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/coinyawong/a